<a href="https://colab.research.google.com/github/ueche/ml-class/blob/main/projects/Ulysses_Echeverria_Bach_Chorales_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Bach Chorales Project
Ulysses Echeverria  
PID:1328350

In [2]:
# datat file: https://raw.githubusercontent.com/zacharski/ml-class/master/data/bach.zip

In [3]:
import pandas as pd
import numpy as np
chordData = pd.read_csv('https://raw.githubusercontent.com/zacharski/ml-class/master/data/bach.zip')
chordData

,choral_ID,event_number,C,C#,D,D#,E,F,F#,G,G#,A,A#,B,bass,meter,chord_label
0,000106b_,1,YES,NO,NO,NO,NO,YES,NO,NO,NO,YES,NO,NO,F,3,F_M
1,000106b_,2,YES,NO,NO,NO,YES,NO,NO,YES,NO,NO,NO,NO,E,5,C_M
2,000106b_,3,YES,NO,NO,NO,YES,NO,NO,YES,NO,NO,NO,NO,E,2,C_M
3,000106b_,4,YES,NO,NO,NO,NO,YES,NO,NO,NO,YES,NO,NO,F,3,F_M
4,000106b_,5,YES,NO,NO,NO,NO,YES,NO,NO,NO,YES,NO,NO,F,2,F_M
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5660,015505b_,105,NO,NO,YES,NO,NO,NO,NO,YES,NO,NO,YES,NO,G,4,G_m
5661,015505b_,106,NO,NO,YES,NO,NO,NO,NO,YES,NO,YES,NO,NO,G,3,G_m
5662,015505b_,107,YES,NO,NO,NO,YES,NO,NO,YES,NO,NO,NO,NO,C,5,C_M
5663,015505b_,108,YES,NO,NO,NO,YES,NO,NO,YES,NO,NO,YES,NO,C,3,C_M


In [4]:
#chech for NaN
chordData.isna().sum().sum()
#drop any row with an NaN value
chordData.dropna(inplace=True)
#drop any rows with chord_label of only one frequency
oneVal_count = chordData.chord_label.value_counts().gt(1)
chordData = chordData.loc[chordData.chord_label.isin(oneVal_count[oneVal_count].index)]
#drop the colimns that are not used to train the model
chordData = chordData.drop(['choral_ID','event_number'], axis = 1)



In [5]:
#separate the datat into a features dataset and a labels dataset
bFeatures = chordData.drop(['chord_label'], axis=1)
bLabels = chordData['chord_label']
bFeatures

,C,C#,D,D#,E,F,F#,G,G#,A,A#,B,bass,meter
0,YES,NO,NO,NO,NO,YES,NO,NO,NO,YES,NO,NO,F,3
1,YES,NO,NO,NO,YES,NO,NO,YES,NO,NO,NO,NO,E,5
2,YES,NO,NO,NO,YES,NO,NO,YES,NO,NO,NO,NO,E,2
3,YES,NO,NO,NO,NO,YES,NO,NO,NO,YES,NO,NO,F,3
4,YES,NO,NO,NO,NO,YES,NO,NO,NO,YES,NO,NO,F,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5660,NO,NO,YES,NO,NO,NO,NO,YES,NO,NO,YES,NO,G,4
5661,NO,NO,YES,NO,NO,NO,NO,YES,NO,YES,NO,NO,G,3
5662,YES,NO,NO,NO,YES,NO,NO,YES,NO,NO,NO,NO,C,5
5663,YES,NO,NO,NO,YES,NO,NO,YES,NO,NO,YES,NO,C,3


In [6]:
#one hot encode the features dataset to convert the categorical data into numerical
one_hot = pd.get_dummies(bFeatures)
bFeatures = one_hot

# I tried using it to improve the accuracy but it actually made ir worse
#from sklearn.preprocessing import OneHotEncoder 
#encoder = OneHotEncoder(
#  categories = 'auto', sparse_output= True)

#bFeatures = encoder.fit_transform(bFeatures)

In [12]:
#fit the labels
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
le.fit(bLabels)
bLabels = le.fit_transform(bLabels)
bLabels

array([75, 34, 34, ..., 34, 34, 75])

In [13]:
#split the data
from sklearn.model_selection import train_test_split
bach_train_features, bach_test_features, bach_train_labels, bach_test_labels = train_test_split(bFeatures, bLabels, test_size = 0.2, random_state=42)


In [16]:
#train the model with the training features and training labels
from xgboost import XGBClassifier
chordData_model = XGBClassifier(tree_method = 'gpu_hist', predictor = 'gpu_predictor', max_depth = 6)

chordData_model.fit(bach_train_features, bach_train_labels)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=6, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=None, num_parallel_tree=None,
              objective='multi:softprob', predictor='gpu_predictor', ...)

In [17]:
#Get the predictions from the trained model with the test features
predictions = chordData_model.predict(bach_test_features)

In [42]:
#from sklearn.metrics import accuracy_score
#training_predictions = chordData_model.predict(bach_train_features)
#training_score = accuracy_score(bach_train_labels, training_predictions)
#training_score



In [18]:
#compute the accuracy of the predictions
from sklearn.metrics import accuracy_score
predictions = chordData_model.predict(bach_test_features)
score = accuracy_score(bach_test_labels, predictions)

score

0.7429328621908127